In [2]:
from db_connect import db_get_df
from segment_ranking.rank_segments import get_most_similar_documents_Bert
from Audio_segmentation.split_audio import produce_snippets
from Audio_segmentation.concat_audio import produce_audio
from transformers import BertModel, BertTokenizer
import json

In [ ]:
df = db_get_df(table="transcript_segments")
print(len(df))
print(df.dtypes)

Wir verwenden hier das Deutsche model, um die deutschen Transkripte optimal zu encodieren.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')
model = BertModel.from_pretrained('bert-base-german-cased', output_hidden_states = True)

In [ ]:
df["tokens"] = [tokenizer.tokenize(text) for text in df["segment_text"]]

In [ ]:
print(df["tokens"].sample(2).to_markdown())

In [3]:
df = db_get_df(table="transcript_segments_MiniLM_L6")

print(len(json.loads(df.loc[0, "embedding_json"])))


384
